In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import xarray as xr
import numpy as np                
import os, subprocess
from shutil import copy2

In [48]:
new = xr.open_mfdataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/r??_template.nc')
ref = xr.open_mfdataset('/Users/robert/Dropbox/Scientific/Projects/RFMIP/RFMIP-IRF/LBLRTM/r??.nc')
new.load()
ref.load()
conds = xr.open_dataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-0-4_none.nc')
conds.load()             
toa = conds.isel(site=0,expt=0).variables["pres_level"].argmin()
sfc = conds.isel(site=0,expt=0).variables["pres_level"].argmax()
(toa, sfc)

(<xarray.Variable ()>
 array(0), <xarray.Variable ()>
 array(60))

In [59]:
print("Fluxes at present day")
print("Mean at toa", (      ref.isel(expt=1,level=toa) * ref.variables["profile_weight"]).sum())
print("Err  at toa", ((new-ref).isel(expt=1,level=toa) * ref.variables["profile_weight"]).sum())
print("Mean at sfc", (      ref.isel(expt=1,level=sfc) * ref.variables["profile_weight"]).sum())
print("Err  at sfc", ((new-ref).isel(expt=1,level=sfc) * ref.variables["profile_weight"]).sum())

Fluxes at present day
Mean at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.01
    rld             float64 0.0
    profile_weight  float64 0.01053
    rlu             float64 266.0
    rsd             float64 335.1
    rsu             float64 47.37
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 -0.1105
    rsd             float64 0.1126
    rsu             float64 0.2275
Mean at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 9.862e+04
    rld             float64 313.3
    profile_weight  float64 0.01053
    rlu             float64 397.8
    rsd             float64 246.1
    rsu             float64 30.21
Err  at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.8184
    profile_weight  floa

In [60]:
#
# Forcing - check defintion (which is PI, PD?)
#
ref_f = ref.isel(expt=1) - ref.isel(expt=0)
new_f = new.isel(expt=1) - new.isel(expt=0)

print("Forcing relative to PI")
print("Mean at toa", (       ref_f.isel(level=toa)  * ref.variables["profile_weight"]).sum())
print("Err  at toa", ((new_f-ref_f).isel(level=toa) * ref.variables["profile_weight"]).sum())
print("Mean at sfc", (       ref_f.isel(level=sfc)  * ref.variables["profile_weight"]).sum())
print("Err  at sfc", ((new_f-ref_f).isel(level=sfc) * ref.variables["profile_weight"]).sum())




Forcing relative to PI
Mean at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 2.845
    rsd             float64 0.01483
    rsu             float64 0.05815
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 -0.562
    rsd             float64 -0.01483
    rsu             float64 -0.007025
Mean at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 -2.083
    profile_weight  float64 0.0
    rlu             float64 -0.04165
    rsd             float64 0.5344
    rsu             float64 0.05609
Err  at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.4276
    profile_weight  flo

In [3]:
print("Range in errrors in longwave fluxes by experiment")
for v in ["rld", "rlu"]: 
  print(v,     (new.variables[v] - ref.variables[v]).min(dim="site").min(dim="level").values) 
  print("   ", (new.variables[v] - ref.variables[v]).max(dim="site").max(dim="level").values)

print("Range in errrors in shortwave fluxes by experiment")
for v in ["rsd", "rsu"]: 
  print(v, 
        (new.isel(expt=[0,1]).variables[v] - ref.isel(expt=[0,1]).variables[v]).min(dim="site").min(dim="level").values) 
  print("   ", 
        (new.isel(expt=[0,1]).variables[v] - ref.isel(expt=[0,1]).variables[v]).max(dim="site").max(dim="level").values)



Range in errrors in longwave fluxes by experiment
rld [-1.8345642 -1.2353821 -1.7793121 -1.8327637 -2.015686  -1.847168
 -1.8350525 -2.1949005 -1.7826538 -1.7366638 -1.8306274 -1.8549194
 -1.3552856 -1.9270935 -1.8694763 -4.086334  -1.8617554 -1.2713013]
    [3.0360718 3.2490845 3.0119324 3.0381165 3.1241455 2.9821777 2.969513
 3.3969727 3.0891724 3.054596  3.0319214 3.0102844 3.205597  3.1357422
 3.6026    2.6705322 3.160797  3.255127 ]
rlu [ -1.6108093  -1.7380066  -1.7139587  -1.6134338  -1.3541565  -1.6278687
  -1.6708374  -2.058838   -1.581665   -1.6083984  -1.6051025  -1.5898743
  -1.8230286 -18.971985  -18.566406  -18.850525  -65.5618     -1.6076355]
    [1.6388245  0.81591797 1.5879517  1.6362     1.699707   1.6339417
 1.6085815  1.375824   1.6331787  1.5379639  1.6146545  1.5990906
 0.87387085 0.06509399 0.07885742 0.05722046 0.06530762 0.7457886 ]
Range in errrors in shortwave fluxes by experiment
rsd [-2.2943726 -2.369751 ]
    [1.7262955 1.7143631]
rsu [-7.986343  -7.950598

In [ ]:
ref.close()
new.close()